In [52]:
from catboost import Pool,CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
from pandas.api.types import is_numeric_dtype
import catboost as cb
from sklearn.metrics import classification_report

In [9]:
df = pd.read_pickle(' all_data.pkl ')
df['buy'] = None
dfno = df.loc[df['dt']==0]
dfye = df.loc[df['dt']>0]
dfye['buy'] = dfye['buy'].apply(lambda x: 1)
dfno['buy'] = dfno['buy'].apply(lambda x: 0)
dfans = pd.concat([dfye,dfno])
dfans

,id,gender,age,education,city,country,lbt_coef,ac_coef,sm_coef,personal_coef,product,colour,cost,product_sex,base_sale,dt,buy
0,0,0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072,"Велосипед горный женский Stern Mira 2.0 26""",белый/синий,13599,0.0,1,7,1
1,0,0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072,Стол Outventure,зелёный,1499,0.0,0,37,1
2,0,0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072,Набор Outventure: стол + 4 стула,бежевый,4799,0.0,0,37,1
5,4,0,35,среднее,1134,32,4.569378,0.063693,0.820892,0.5072,Роликовые коньки детские Roces Yuma TIF,мультицвет,6999,0.0,0,1,1
6,4,0,35,среднее,1134,32,4.569378,0.063693,0.820892,0.5072,Набор защиты из 3-х предметов детский Reaction...,белый/серый,999,0.0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785972,102515,1,18,высшее,1179,32,4.174581,0.142444,1.201523,0.2896,Кеды для мальчиков adidas Grand Court K,Unknown,2499,1.0,0,0,0
786179,143729,1,27,среднее,1185,32,7.488824,-0.258205,1.248983,0.4304,Брюки мужские Outventure,темно-синий,1999,1.0,0,0,0
786217,130605,1,36,среднее,1189,32,7.766175,0.015780,0.764295,0.4304,Мяч футбольный Puma Laliga 1 Accelerate Hyb,белый/розовый/голубой,3499,1.0,0,0,0
786248,15885,1,33,среднее,1190,32,7.321749,0.470877,0.699928,0.4304,Кеды мужские PUMA Smash PERF,черный/серый/белый,3749,1.0,1,0,0


In [25]:
dfans_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600000 entries, 0 to 615621
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             600000 non-null  int64  
 1   gender         600000 non-null  int64  
 2   age            600000 non-null  int64  
 3   education      600000 non-null  object 
 4   city           600000 non-null  int64  
 5   country        600000 non-null  int64  
 6   lbt_coef       600000 non-null  float64
 7   ac_coef        600000 non-null  float64
 8   sm_coef        600000 non-null  float64
 9   personal_coef  600000 non-null  float64
 10  product        600000 non-null  object 
 11  colour         600000 non-null  object 
 12  cost           600000 non-null  int64  
 13  product_sex    600000 non-null  float64
 14  base_sale      600000 non-null  int64  
 15  dt             600000 non-null  int64  
 16  buy            600000 non-null  int64  
dtypes: float64(5), int64(9), object(3)

In [44]:
X = dfans.drop('buy', axis=1)
y = dfans['buy']

In [53]:
def get_categorical_indicies(X):
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        cat_indicies.append(X.columns.get_loc(col))
    return cat_indicies
categorical_indicies = get_categorical_indicies(X)

def convert_cats(X):
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        X[col] = X[col].astype('category')
convert_cats(X)

In [54]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [55]:

train_dataset = cb.Pool(X_train,y_train,
                        cat_features=categorical_indicies)                                                     
test_dataset = cb.Pool(X_test,y_test,          
                       cat_features=categorical_indicies)
                      

In [56]:
model = cb.CatBoostClassifier(loss_function='Logloss', 
                              eval_metric='Accuracy')
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5,],
        'iterations': [50, 100, 150]}

In [57]:
model.grid_search(grid,train_dataset)
pred = model.predict(X_test)
print(classification_report(y_test, pred))

0:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 356ms	remaining: 17.4s
1:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 618ms	remaining: 14.8s
2:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 776ms	remaining: 12.2s
3:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 975ms	remaining: 11.2s
4:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 1.11s	remaining: 10s
5:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 1.28s	remaining: 9.39s
6:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 1.43s	remaining: 8.76s
7:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 1.59s	remaining: 8.33s
8:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 1.75s	remaining: 7.96s
9:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 1.93s	remaining: 7.71s
10:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 2.09s	remaining: 7.4s
11:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)